In [65]:
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Flatten, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.optimizers import rmsprop


In [66]:
import os 
def ReadAllData(rootDir): 
    
    all_data = []
    for class_idx in range(2):
        data_path  = rootDir + "/" + str(class_idx+1) + "/"
        for lists in os.listdir(data_path): 
            path = os.path.join(data_path, lists) 
            if os.path.isdir(path): 
                print("This is dir")
                continue
            one_data = {}
            if path[-4:] != ".bmp":
                print(path)
                continue
            one_data["file_name"] = path
            one_data["label"] = class_idx + 1
            all_data.append(one_data)
    return all_data

In [67]:
all_data = ReadAllData("TestImage1")
all_data += ReadAllData("TestImage2")
# all_data += ReadAllData("../lostLine/lostLine1")
print(len(all_data))

TestImage1/1/.DS_Store
TestImage1/2/.DS_Store
TestImage2/1/.DS_Store
TestImage2/2/.DS_Store
3625


In [68]:
import numpy as np
val_split_ratio = 0.9 
np.random.shuffle(all_data)
val_split = int(val_split_ratio * len(all_data))
train_data_list = all_data[0:val_split]
val_data_list = all_data[val_split:]

In [85]:
import cv2
def get_batch(index, size, train_type):
    x_data = []
    
    y_data= []
    if train_type == 0: 
        #Read train data
        np.random.shuffle(train_data_list)
        for idx in range(size):
            image_idx = (index+idx)%len(train_data_list)
            image_name = train_data_list[image_idx]["file_name"]
            
            image_label = train_data_list[image_idx]["label"]
            #print(image_name,image_label)
            src_image = cv2.imread(image_name)
            #gray_image = cv2.cvtColor(src_image,cv2.COLOR_BGR2GRAY)
            src_image = cv2.resize(src_image,(80,30),interpolation=cv2.INTER_CUBIC)
            input_data = src_image
            
            image_label_one_hot = False
            if image_label == 1:
                image_label_one_hot = False
            else:
                image_label_one_hot = True
            x_data.append(input_data)
            y_data.append(image_label_one_hot)
    else:
        #Reade val data
        np.random.shuffle(val_data_list)
        for idx in range(size):
            image_idx = (index+idx)%len(val_data_list)
            image_name = val_data_list[image_idx]["file_name"]
            image_label = val_data_list[image_idx]["label"]
            src_image = cv2.imread(image_name)
            #gray_image = cv2.cvtColor(src_image,cv2.COLOR_BGR2GRAY)
            src_image = cv2.resize(src_image,(80,30),interpolation=cv2.INTER_CUBIC)
            input_data = src_image
            
            image_label_one_hot = False
            if image_label == 1:
                image_label_one_hot = False
            else:
                image_label_one_hot = True
            x_data.append(input_data)
            y_data.append(image_label_one_hot)
    
    return (np.array(x_data), np.array(y_data))
test_data = get_batch(0,2,0)
print(test_data[0].shape)


(2, 30, 80, 3)


In [86]:
cur_train_index = 0
minibatch_size = 16
def next_train():
    global cur_train_index
    while 1:
        ret = get_batch(cur_train_index, minibatch_size, train_type=0)
        cur_train_index += minibatch_size
        if cur_train_index >= len(train_data_list):
            cur_train_index = cur_train_index%minibatch_size
        yield ret

In [87]:
cur_val_index = 0
def next_val():
    global cur_val_index
    while 1:
        ret = get_batch(cur_val_index, minibatch_size, train_type=1)
        cur_val_index += minibatch_size
        if cur_val_index >= len(val_data_list):
            cur_val_index = cur_val_index%minibatch_size
        yield ret

In [88]:
from keras.optimizers import SGD
def train_LeNet(input_shape):
    # Network parameters
    act = 'relu'

    input_data = Input(name='the_input', shape=input_shape, dtype='float32')

    inner = Conv2D(32, (5,5), padding='valid',
                   activation=act, kernel_initializer='he_normal',
                   name='conv1')(input_data)
    inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)
    inner = Conv2D(32, (5,5), padding='valid',
                   activation=act, kernel_initializer='he_normal',
                   name='conv2')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)
    
    inner = Conv2D(32, (3,3), padding='valid',
                   activation=act, kernel_initializer='he_normal',
                   name='conv3')(inner)
    
    inner = Flatten(name='flatten1')(inner)
    
    inner = Dense(120, activation=act, name='dense1')(inner)
    
    inner = Dense(84, activation=act, name='dense2')(inner)
    
    inner = Dense(1, activation=act, name='dense3')(inner)
    
    y_pred = Activation('softmax', name='softmax')(inner)
    
    model = Model(inputs=input_data, outputs=y_pred, name='lenet_5')
    
    model.summary()
    
    return model
my_LeNet = train_LeNet((30,80,3))    
minibatch_size = 32
my_LeNet.compile(
    loss='categorical_crossentropy',
    #optimizer=SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
    optimizer = rmsprop(lr=0.0001),
    metrics=['accuracy']
    )

my_LeNet.fit_generator(generator=next_train(),
                        steps_per_epoch=400,  #400
                        epochs=5,
                        validation_data=next_val(),
                        validation_steps=50)
    


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       (None, 30, 80, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 26, 76, 32)        2432      
_________________________________________________________________
max1 (MaxPooling2D)          (None, 13, 38, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 9, 34, 32)         25632     
_________________________________________________________________
max2 (MaxPooling2D)          (None, 4, 17, 32)         0         
_________________________________________________________________
conv3 (Conv2D)               (None, 2, 15, 32)         9248      
_________________________________________________________________
flatten1 (Flatten)           (None, 960)               0         
__________

ValueError: You are passing a target array of shape (32, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils.np_utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.

In [40]:
my_LeNet.fit_generator(generator=next_train(),
                        steps_per_epoch=400,  #400
                        epochs=5,
                        validation_data=next_val(),
                        validation_steps=50)

Epoch 1/5
400/400 [==============================] - 71s - loss: 0.1239 - acc: 0.9456 - val_loss: 0.2123 - val_acc: 0.9387
Epoch 2/5
400/400 [==============================] - 68s - loss: 0.1102 - acc: 0.9586 - val_loss: 0.2627 - val_acc: 0.9525
Epoch 3/5
400/400 [==============================] - 69s - loss: 0.1222 - acc: 0.9556 - val_loss: 0.2092 - val_acc: 0.8531
Epoch 4/5
400/400 [==============================] - 68s - loss: 0.0826 - acc: 0.9717 - val_loss: 0.1925 - val_acc: 0.9619
Epoch 5/5
400/400 [==============================] - 68s - loss: 0.0556 - acc: 0.9830 - val_loss: 0.1461 - val_acc: 0.9819


In [42]:
my_LeNet.fit_generator(generator=next_train(),
                        steps_per_epoch=400,  #400
                        epochs=5,
                        validation_data=next_val(),
                        validation_steps=50)

Epoch 1/5
400/400 [==============================] - 76s - loss: 0.0746 - acc: 0.9802 - val_loss: 0.0339 - val_acc: 0.9938
Epoch 2/5
400/400 [==============================] - 101s - loss: 0.0452 - acc: 0.9877 - val_loss: 0.0551 - val_acc: 0.9800
Epoch 3/5
400/400 [==============================] - 97s - loss: 0.0471 - acc: 0.9866 - val_loss: 0.0251 - val_acc: 0.9881
Epoch 4/5
400/400 [==============================] - 106s - loss: 0.0554 - acc: 0.9880 - val_loss: 0.0314 - val_acc: 0.9925
Epoch 5/5
400/400 [==============================] - 109s - loss: 0.0276 - acc: 0.9931 - val_loss: 0.2761 - val_acc: 0.9831


In [43]:
my_LeNet.save("lostLine_v3.h5")

In [ ]:
my_LeNet.fit_generator(generator=next_train(),
                        steps_per_epoch=400,  #400
                        epochs=5,
                        validation_data=next_val(),
                        validation_steps=50)

In [15]:
import time
test_data = get_batch(0,1,0)
real_resul = test_data[1]
start_time = time.time()
predict_result = my_LeNet.predict(test_data[0])
end_time = time.time()
print("real_resul", np.argmax(real_resul))
print("predict_result", np.argmax(predict_result))
print("cost time",end_time - start_time)

real_resul 0
predict_result 0
cost time 0.0037949085235595703


In [14]:
test_data = get_batch(0,1,0)
real_resul = test_data[1]
start_time = time.time()
predict_result = my_LeNet.predict(test_data[0])
end_time = time.time()
print("real_resul", np.argmax(real_resul))
print("predict_result", np.argmax(predict_result))
print("cost time",end_time - start_time)

real_resul 0
predict_result 0
cost time 0.0045239925384521484


In [ ]:
my_LeNet.save("myLeNet.h5")

In [149]:
from keras.models import load_model
load_LeNet = load_model('myLeNet.h5')

In [152]:
test_data = get_batch(0,1,0)
real_resul = test_data[1]
start_time = time.time()
predict_result = load_LeNet.predict(test_data[0])
end_time = time.time()
print("real_resul", np.argmax(real_resul))
print("predict_result", np.argmax(predict_result))
print("cost time",end_time - start_time)

real_resul 1
predict_result 1
cost time 0.005012989044189453


In [138]:
start_time = time.time()
predict_result = load_LeNet.predict(test_data[0])
end_time = time.time()
print("real_resul", np.argmax(real_resul))
print("predict_result", np.argmax(predict_result))
print("cost time",end_time - start_time)

real_resul 2
predict_result 2
cost time 0.005013704299926758


In [130]:
# minibatch_size = 16
# my_LeNet.compile(
#     loss='categorical_crossentropy',
#     #optimizer=SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
#     optimizer = rmsprop(lr=0.0001),
#     metrics=['accuracy']
#     )

# my_LeNet.fit_generator(generator=next_train(),
#                         steps_per_epoch=400,  #400
#                         epochs=5,
#                         validation_data=next_val(),
#                         validation_steps=50)

Epoch 1/5


KeyboardInterrupt: 

In [118]:
my_LeNet.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
    metrics=['accuracy']
    )

my_LeNet.fit_generator(generator=next_train(),
                        steps_per_epoch=400,  #400
                        epochs=5,
                        validation_data=next_val(),
                        validation_steps=50)

Epoch 1/5
273/400 [===================>..........] - ETA: 7s - loss: 2.0794 - acc: 0.1445

KeyboardInterrupt: 

In [93]:
import time
test_data = get_batch(0,1,0)
real_resul = test_data[1]
start_time = time.time()
predict_result = my_LeNet.predict(test_data[0])
end_time = time.time()
print("real_resul", np.argmax(real_resul))
print("predict_result", np.argmax(predict_result))
print("cost time",end_time - start_time)

real_resul 2
predict_result 2
cost time 0.0050127506256103516
